#### ex_13_2 RNN

根据名字识别他所在的国家

人名字符长短不一，最长的10个字符，所以处理成10维输入张量，都是英文字母刚好可以映射到ASCII上

Maclean ->  ['M', 'a', 'c', 'l', 'e', 'a', 'n'] ->  [ 77 97 99 108 101 97 110]  ->  [ 77 97 99 108 101 97 110 0 0 0]

共有18个国家，设置索引为0-17

训练集和测试集的表格文件都是第一列人名，第二列国家

In [1]:
import torch
import  time
import csv
import gzip
from  torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import datetime
import matplotlib.pyplot as plt
import numpy as np
import random
 
# Parameters
HIDDEN_SIZE = 100
BATCH_SIZE = 256
N_LAYER = 2
N_EPOCHS = 100
N_CHARS = 128
USE_GPU = True

# 初始化并固定随机种子


def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


setup_seed(1012)

# 设置GPU加速
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The current computing device is {device.type} ")
if torch.cuda.is_available():
    print(f'The current GPU is :{torch.cuda.get_device_name(0)}')

def create_tensor(tensor):  #如果使用GPU，把tensor搬到GPU上去
    tensor = tensor.to(device)
    return tensor


The current computing device is cpu 


In [3]:
class NameDataset(Dataset):         #处理数据集
    def __init__(self, is_train_set=True):
        filename = 'names_train.csv.gz' if is_train_set else 'names_test.csv.gz'
        with gzip.open(filename, 'rt') as f:    #打开压缩文件并将变量名设为为f
            reader = csv.reader(f)              #读取表格文件
            rows = list(reader)
        self.names = [row[0] for row in rows]               # 所有数据集中的姓名
        self.countries = [row[1] for row in rows]           # 所有数据集中的国家
        self.len = len(self.names)                      # 数据集总长度
        self.country_list = list(set(self.countries))   # 保存所有国家名（各一次） 索引与值正好与词典对偶
        self.country_num = len(self.country_list)   # 所有国家的类别总数
        self.country_dict = self.getCountryDict()   # 返回一个词典，键：国家名 值：数字
        
    def __getitem__(self, index):
        return self.names[index], self.countries_dict[self.countries[index]]
 
    def __len__(self):
        return self.len
 
    def getCountryDict(self):
        country_dict = dict()                           #创建空字典
        for idx, country in enumerate(self.country_list,0):
            country_dict[country] = idx # 或者 len(country_dict)
        return country_dict
 
    def idx2country(self,index):            #由国家代表的值返回对应的国家名
        return self.country_list[index]
 
    def getCountrysNum(self):               #返回国家类别数量
        return self.country_num

train_set = NameDataset(is_train_set=True)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE,shuffle=True)
test_set = NameDataset(is_train_set=False)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=False)
N_COUNTRY = train_set.getCountrysNum()  # 这也是最终输出的维度(每个预测类别对应一个维度)

In [ ]:
class RNNBaseClassifier(nn.Module): # 基于RNN的分类器（最后会连上一个全连接层）
    def __init__(self,input_size,hidden_size,output_size,num_layers=1,bidirectional= True) -> None:
        super(RNNBaseClassifier,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.n_directions = 2 if bidirectional else 1
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.gru = nn.GRU()